In [2]:
!pip install groq
!pip install python-dotenv

# 1. Minimal Agent

This is our simplest “agent” using groq-api:

* Has a role
* Takes input
* Produces reasoning-based output

In [5]:
import os

from groq import Groq
from dotenv import load_dotenv

load_dotenv()
client = Groq(api_key=os.environ.get("GROQ_API_KEY"))

class SimpleAgent:
    def __init__(self, system_prompt):
        self.system_prompt = system_prompt

    def run(self, user_input):
        response = client.chat.completions.create(
            model="llama-3.3-70b-versatile",
            messages=[
                {"role": "system", "content": self.system_prompt},
                {"role": "user", "content": user_input}
            ],
            temperature=0.2
        )
        return response.choices[0].message.content


agent = SimpleAgent(
    system_prompt="You are a helpful AI agent that reasons step by step."
)

print(agent.run("Explain Kafka vs API based communication"))

To explain Kafka vs API-based communication, let's break down each concept and then compare them.

**API-based Communication:**

API (Application Programming Interface) is a set of defined rules that enable different applications, systems, or services to communicate with each other. It's a request-response model where a client (e.g., a web application) sends a request to a server, and the server responds with the requested data.

Here's a step-by-step overview of API-based communication:

1. **Request**: The client sends an HTTP request (e.g., GET, POST, PUT, DELETE) to the server with the required data.
2. **Server Processing**: The server processes the request, performs the necessary actions, and retrieves the required data.
3. **Response**: The server sends an HTTP response back to the client with the requested data.

API-based communication is widely used for real-time communication between applications, such as:

* Web applications communicating with backend servers
* Mobile apps 

# 2. Stateful Agent (Memory + Iterative Reasoning)

Now this agent:

* Remembers context
* Builds conversation state
* Can ask clarifying questions


### Response from the llm:

```
ChatCompletion(
    id='chatcmpl-c4554bf1-edd2-4c87-98c1-e4595aa5d831', 
    choices=[
        Choice(
            finish_reason='stop', 
            index=0, 
            logprobs=None, 
            message=ChatCompletionMessage(
                content="To help you decide between Kafka and API, ... and I'll help you make an informed decision!", 
                role='assistant', 
                annotations=None, 
                executed_tools=None, 
                function_call=None, 
                reasoning=None, 
                tool_calls=None
            )
        )
    ], 
    created=1769932176, 
    model='llama-3.3-70b-versatile', 
    object='chat.completion', 
    mcp_list_tools=None, 
    service_tier='on_demand', 
    system_fingerprint='fp_43d97c5965', 
    usage=CompletionUsage(
        completion_tokens=340, 
        prompt_tokens=56, 
        total_tokens=396, 
        completion_time=1.149168506, 
        completion_tokens_details=None, 
        prompt_time=0.002955258, 
        prompt_tokens_details=None, 
        queue_time=0.008930827, 
        total_time=1.152123764
    ), 
    usage_breakdown=None, 
    x_groq=XGroq(
        id='req_01kgc2ve1segyrev1pmj1b891k', 
        debug=None, 
        seed=831061670, 
        usage=None
    )
)
```

In [12]:
from groq import Groq
from pprint import pprint

client = Groq(api_key=os.environ.get("GROQ_API_KEY"))

class MemoryAgent:
    def __init__(self, system_prompt):
        self.messages = [{"role": "system", "content": system_prompt}]

    def run(self, user_input):
        self.messages.append({"role": "user", "content": user_input})

        response = client.chat.completions.create(
            model="llama-3.3-70b-versatile",
            messages=self.messages,
            temperature=0.2
        )

        assistant_msg = response.choices[0].message.content
        self.messages.append({"role": "assistant", "content": assistant_msg})
        return assistant_msg


agent = MemoryAgent(
    system_prompt="You are a senior system design agent. Ask clarifying questions when needed."
)

print(agent.run("Help me choose Kafka vs API"))
print(agent.run("Downstream SLA is 4 hours\n"))
print(f"agent workflow as chat: \n")
pprint(agent.messages)

To help you choose between Kafka and API, I need to understand your use case a bit better. Can you please provide some context about your project? Here are a few clarifying questions to get started:

1. What type of data are you trying to handle (e.g., real-time events, batch processing, request-response)?
2. What is the volume and velocity of the data (e.g., how much data, how often)?
3. Are you building a new system or integrating with existing systems?
4. What are your primary concerns (e.g., scalability, reliability, latency, security)?
5. Are there any specific technologies or frameworks you're already using or planning to use?

Kafka and API are designed for different purposes:

* Kafka is a distributed streaming platform designed for high-throughput, fault-tolerant, and scalable data processing. It's often used for event-driven architectures, real-time data processing, and microservices communication.
* API (Application Programming Interface) is a set of defined rules that enabl

# 3. Tool-Calling Agent (True “Agentic” Behavior)

Agent decides whether to call a tool or think. We can then:

* Parse structured JSON
* Holds memory (messages)
* Registers tools
* Runs agent loop (Loop until task completes)
* Executes tools
* Returns final answer

In [15]:
import json
from groq import Groq


class ToolAgent:
    def __init__(self, model="llama-3.3-70b-versatile"):
        self.client = Groq()
        self.model = model

        self.messages = [
            {
                "role": "system",
                "content": (
                    "You are a senior system design agent.\n"
                    "- If information is missing, ask clarifying questions.\n"
                    "- If enough information is available, call the tool ONCE.\n"
                    "- After receiving tool output, provide a FINAL answer.\n"
                    "- Do NOT call tools again after a tool result."
                )
            }
        ]

        self.tools = [
            {
                "type": "function",
                "function": {
                    "name": "choose_transport",
                    "description": "Decide Kafka vs API based on system constraints",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "sla_hours": {"type": "integer"},
                            "volume": {"type": "integer"},
                            "push": {"type": "boolean"}
                        },
                        "required": ["sla_hours", "volume", "push"]
                    }
                }
            }
        ]

        self.tool_used = False  # 🔒 critical flag

    # --------------------
    # Tool implementation
    # --------------------
    def choose_transport(self, sla_hours: int, volume: int, push: bool) -> dict:
        if push and sla_hours >= 2 and volume >= 1000:
            return {
                "decision": "Kafka",
                "reason": "High volume, async SLA, push-based architecture"
            }
        return {
            "decision": "API",
            "reason": "Lower volume or synchronous requirement"
        }

    def _execute_tool(self, name: str, args: dict) -> dict:
        if name == "choose_transport":
            return self.choose_transport(**args)
        raise ValueError(f"Unknown tool: {name}")

    # --------------------
    # Agent loop
    # --------------------
    def run(self, user_input: str, max_steps: int = 5) -> str:
        self.messages.append({"role": "user", "content": user_input})

        for step in range(max_steps):
            response = self.client.chat.completions.create(
                model=self.model,
                messages=self.messages,
                tools=self.tools if not self.tool_used else None,
                tool_choice="auto" if not self.tool_used else "none",
                temperature=0
            )

            message = response.choices[0].message

            # 🔧 Tool call (only once)
            if message.tool_calls and not self.tool_used:
                self.tool_used = True

                tool_call = message.tool_calls[0]
                tool_name = tool_call.function.name
                tool_args = json.loads(tool_call.function.arguments)

                print(f"\n🔧 Tool called: {tool_name}({tool_args})")

                result = self._execute_tool(tool_name, tool_args)

                self.messages.append({
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "content": json.dumps(result)
                })

            # ✅ Final answer
            else:
                self.messages.append({
                    "role": "assistant",
                    "content": message.content
                })
                return message.content

        raise RuntimeError("Agent did not converge within max steps")

In [16]:
agent = ToolAgent()

output = agent.run(
    "Push-based system, thousands of comparisons per run, "
    "downstream SLA is 4 hours"
)

print("\n✅ FINAL OUTPUT:\n", output)


🔧 Tool called: choose_transport({'push': True, 'sla_hours': 4, 'volume': 1000})

✅ FINAL OUTPUT:
 FINAL ANSWER: Based on the provided information, the recommended system design is Kafka. This is because Kafka is well-suited for handling high volumes of data, has an asynchronous SLA, and is designed for push-based architectures, making it a good fit for the described use case with thousands of comparisons per run and a downstream SLA of 4 hours.
